# Compute $P_{\mathrm{O}_2}$ from drift corrected fields

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
from itertools import product
import yaml
import intake
import numpy as np
import xarray as xr

import funnel
import data_collections as dc
import util

In [3]:
try:
    cluster
    client
except:
    cluster, client = util.get_ClusterClient(memory='100GB')
    cluster.scale(34)
client    

Connection method: Cluster object,Cluster type: PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.35:44434,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
sub_spec = dict(
    name='drift-corrected',
    experiment=['20C', 'RCP85'],
    member_id=dc.ocean_bgc_member_ids[1:], # first time level for ensemble member 1 different
)

catalog = funnel.to_intake_esm(agg_member_id=False).search(**sub_spec)
catalog.df

,experiment,component,stream,member_id,variable,name,path
0,20C,ocn,pop.h,32,SALT,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
1,RCP85,ocn,pop.h,2,TEMP,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
2,20C,ocn,pop.h,30,TEMP,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
3,RCP85,ocn,pop.h,35,TEMP,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
4,20C,ocn,pop.h,102,TEMP,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
...,...,...,...,...,...,...,...
259,RCP85,ocn,pop.h,101,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
260,RCP85,ocn,pop.h,102,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
261,RCP85,ocn,pop.h,103,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
262,RCP85,ocn,pop.h,104,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...


In [5]:
catalog.search(experiment='RCP85', member_id=10).df

,experiment,component,stream,member_id,variable,name,path
0,RCP85,ocn,pop.h,10,TEMP,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
1,RCP85,ocn,pop.h,10,SALT,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
2,RCP85,ocn,pop.h,10,O2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...
3,RCP85,ocn,pop.h,10,pO2,drift-corrected,/glade/scratch/mclong/ocean-metabolism/funnel-...


In [6]:
cache_dir = '/glade/scratch/mclong/ocean-metabolism/funnel-cache'

def _gen_cache_file_name(experiment, component, stream, member_id, variable):
    return f'{cache_dir}/glade-cesm1-le.{experiment}.{component}.{stream}.{int(member_id):03d}.{variable}.drift-corrected.zarr'


def _make_cache(experiment, component, stream, member_id, variable):
    """
    Manually generate funnel catalog entry
    
    I.e.:
    asset: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.101.TEMP.drift-corrected.zarr
    esm_collection: data/catalogs/glade-cesm1-le.json
    key: 20C.ocn.pop.h.101
    name: drift-corrected
    operator_kwargs:
    - {}
    - {}
    - {}
    operators:
    - compute_time
    - sel_time_slice
    - compute_drift_correction
    preprocess: _preprocess_pop_h_upper_1km
    variable: TEMP
    """
    
    cache_id_dict = dict(
        asset=_gen_cache_file_name(experiment, component, stream, member_id, variable),
        esm_collection='data/catalogs/glade-cesm1-le.json',        
        key=f'{experiment}.{component}.{stream}.{member_id}',
        name='drift-corrected',
        operator_kwargs=[{}, {}, {}],
        operators=['compute_time', 'sel_time_slice', 'compute_drift_correction'],
        preprocess='_preprocess_pop_h_upper_1km',
        variable=variable,
    )
    cache_id_file = f'data/funnel-catalog/glade-cesm1-le.{experiment}.{component}.{stream}.{int(member_id):03d}.{variable}.drift-corrected.yml'
    with open(cache_id_file, 'w') as fid:
        yaml.dump(cache_id_dict, fid)    

In [7]:
clobber = False

experiment_list = catalog.unique('experiment')['experiment']['values']
member_id_list = catalog.unique('member_id')['member_id']['values']

stream = 'pop.h'
component = 'ocn'

# refresh catalog
catalog = funnel.to_intake_esm(agg_member_id=False).search(**sub_spec)

for experiment, member_id in product(experiment_list, member_id_list):
    cat = catalog.search(
        experiment=experiment, 
        member_id=member_id, 
        stream=stream, 
        component=component,
        variable=['TEMP', 'SALT', 'O2'],
    )
    
    # ensure variables
    missing_vars = {'TEMP', 'SALT', 'O2'} - set(cat.df.variable.to_list())
    if missing_vars:
        print(f'missing vars for {experiment}.{member_id:03d}: {missing_vars}')
        continue

    # check for existing cache file
    variable = 'pO2'
    asset = _gen_cache_file_name(experiment, component, stream, member_id, variable)
    
    if clobber and os.path.exists(asset):
        print(f'removing: {asset}')
        shutil.rmtree(asset)
        
    if os.path.exists(asset):
        print(f'exists: {asset}')
        continue    
    
    # compute pO2
    print(f'computing/writing: {asset}')
    dset = cat.to_dataset_dict()
    assert len(dset.keys()) == 1    
    _, ds = dset.popitem()
    
    ds = dc.compute_pO2(ds)
    ds.to_zarr(asset, mode='w', consolidated=True)    
    
    _make_cache(experiment, component, stream, member_id, variable)    
    

exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.002.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.009.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.010.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.011.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.012.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.013.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.014.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-cache/glade-cesm1-le.20C.ocn.pop.h.015.pO2.drift-corrected.zarr
exists: /glade/scratch/mclong/ocean-metabolism/funnel-ca

In [9]:
client.close()
cluster.close()